В папке subsid (shared/homeworks/python_ds_miniprojects/5_subsid) находятся файлы (tm_sales_1, tm_sales_2, ...)
с продажами продуктов через телемаркетинг. 
Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, 
соединив файлы о продажах с логами по подключениям в системе.

Особенности данных:

сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.


In [2]:
import os
import pandas as pd

url = '/home/jupyter-n.pjatibratova-8/shared/homeworks/python_ds_miniprojects/5_subsid'

all_data = pd.DataFrame()
current_data = pd.DataFrame()

dirs = os.walk(url)
for path_from_top, subdirs, files in dirs:
    for f in files:
        if f.endswith('csv'):
            if f != 'prod_activations_logs.csv':
                path_name = path_from_top + '/' + f
                fd = pd.read_csv(path_name, sep=';')
                all_data = all_data.append(fd).reset_index(drop=True)
#если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
all_data = all_data.dropna(subset = ['SUBS_ID'])            
all_data

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,8362218,7,9879,05-03-2020 11:42
6,2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [3]:
# если 'id' нет в начале SUBS_ID, то нужно его добавить
def append_id(value):
    if value.startswith("id"):
        return value
    return "id" + value
all_data.SUBS_ID = all_data.SUBS_ID.apply(append_id)


In [4]:
all_data.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,id8362218,7,9879,05-03-2020 11:42


In [5]:
# Объедините (inner merge) его с датафрэймом со всеми продажами по колонкам SUBS_ID и PROD_ID
log_url = '/home/jupyter-n.pjatibratova-8/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv'
log_data = pd.read_csv(log_url, sep=';')
log_data

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [6]:
log_data['START_DTTM'] = pd.to_datetime(log_data['START_DTTM'], errors='coerce')
log_data['END_DTTM'] = pd.to_datetime(log_data['END_DTTM'], errors='coerce')
log_data.dtypes

SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [7]:
all_data['ACT_DTTM'] = pd.to_datetime(all_data['ACT_DTTM'], errors='coerce')
all_data.dtypes

SUBS_ID              object
FILIAL_ID             int64
PROD_ID               int64
ACT_DTTM     datetime64[ns]
dtype: object

In [8]:
full_data = all_data.merge(log_data, on = ['SUBS_ID', 'PROD_ID'])
full_data.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-01-12 00:00:00
1,id7646509,5,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00
3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-01-12 00:00:00
4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00


In [9]:
full_data.dtypes

SUBS_ID               object
FILIAL_ID              int64
PROD_ID                int64
ACT_DTTM      datetime64[ns]
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [10]:
full_data['tt'] = full_data['END_DTTM'] - full_data['START_DTTM'] 
full_data

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,tt
0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-01-12 00:00:00,-69 days +09:01:00
1,id7646509,5,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-01-12 00:00:00,-65 days +07:32:00
4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [11]:
data_than_5m = full_data.tt > '5m'

In [12]:
df = full_data.loc[data_than_5m]
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,tt
2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [13]:
df.to_csv()

',SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,tt\n2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00\n4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00\n'

In [14]:
df.SUBS_ID

2    id5416547
4    id7642700
Name: SUBS_ID, dtype: object